# 🐭 &nbsp; <font color="#6DA4AA">Electroencephalogram of Sleeping Mice - Data Exploration</font>

In this practical work, we will explore a dataset about mice's sleep obtained from CHUV. This dataset includes the Electroencephalogram (EEG) of a mouse, as well as its state.

In the long run the objective would be to be able to recognize mice sleep stages from their EEG.

![Sleeping mouse](https://www.cottagesmallholder.com/wp-content/images/mouse-asleep-with-tiny-teddy-bear.jpg)

EEG is a scientific tool that helps us study brain activity. It records the electrical signals in the brain over time, creating a time serie with detailed patterns. These patterns include things like alpha oscillations suggesting a relaxed but awake state, beta oscillations that might indicate active thinking and focus, or theta oscillations typically observed during deep sleep and non-REM sleep stages.

As a reminder, the frequency in Hertz is the number of cycles per second:

![Frequency](https://cmtext.indiana.edu/acoustics/img/4cps.png)

The problem is that the information about these patterns is spread out over many data points, making it hard to detect directly the main frequency of such oscillations.

So, we use something called spectral analysis, which transforms these time series into a different view called the frequency domain. This helps us see the main patterns more clearly by concentrating information about one frequency of oscillation. The process of converting data from the time-domain to the frequency-domain is done thanks to Fourier transforms.

See this diagram to better understand the relationship between the time domain and the frequency domain:
![Frequency Domain](https://www.dataphysics.com/wp-content/uploads/2022/01/The_Frequency_Domain_2.jpg)

Let's look a bit closer at our data. A row was generated each 4 seconds (=15 by minute) during one entire day, giving a dataset of 15x60x24=21600 datapoints. After removing outliers, we are left with 20258 datapoints. For each of this datapoint, the mouse is either:

- awake (**w**)
- in REM sleep (**r**)
- in non-REM sleep (**n**).

In addition to its current state, each datapoint (row) is characterized with a "spectrum". As mentioned earlier, it is really difficult to use the raw electrical activity of the mouse brain. This is why we will focus on the spectrum in order to predict the state of the mouse. The spectrum is described as the amplitude of the signal at each frequency between 1 and 101 [Hz] during the last 4 seconds (=since the last datapoint).

The main point to understand is that at each timestamp (= row = datapoint), the mouse is in a specific state and its brain electrical activity is represented by 101 different features based on frequencies.

In [ ]:
import pandas as pd

mice_eeg = pd.read_csv('EEG_mouse_data.csv')
mice_eeg

,state,amplitude_around_1_Hertz,amplitude_around_2_Hertz,amplitude_around_3_Hertz,amplitude_around_4_Hertz,amplitude_around_5_Hertz,amplitude_around_6_Hertz,amplitude_around_7_Hertz,amplitude_around_8_Hertz,amplitude_around_9_Hertz,...,amplitude_around_92_Hertz,amplitude_around_93_Hertz,amplitude_around_94_Hertz,amplitude_around_95_Hertz,amplitude_around_96_Hertz,amplitude_around_97_Hertz,amplitude_around_98_Hertz,amplitude_around_99_Hertz,amplitude_around_100_Hertz,amplitude_around_101_Hertz
0,n,0.000009,0.000014,0.000013,0.000014,0.000023,0.000009,0.000019,0.000011,0.000016,...,8.364060e-09,4.208368e-09,2.177679e-09,4.342505e-09,5.130010e-09,2.852361e-09,9.772395e-10,1.876307e-09,5.457793e-09,8.653900e-08
1,n,0.000010,0.000010,0.000011,0.000015,0.000022,0.000017,0.000027,0.000011,0.000011,...,6.695745e-09,1.213822e-08,2.287185e-08,3.219380e-09,2.461736e-09,2.985953e-09,1.434412e-09,1.579590e-09,5.458902e-09,8.492570e-08
2,n,0.000011,0.000077,0.000034,0.000019,0.000013,0.000006,0.000010,0.000004,0.000006,...,1.105705e-08,3.264479e-09,1.204057e-09,5.090074e-09,5.477467e-09,1.809223e-09,7.064289e-10,1.435585e-09,7.416121e-09,1.156240e-07
3,w,0.000003,0.000031,0.000044,0.000016,0.000007,0.000006,0.000005,0.000007,0.000006,...,2.223992e-08,1.028462e-08,3.561479e-09,3.704753e-09,2.425319e-09,1.536834e-09,2.540238e-10,6.300358e-10,3.647454e-09,2.212800e-08
4,w,0.000010,0.000011,0.000012,0.000017,0.000020,0.000018,0.000014,0.000025,0.000018,...,6.541705e-09,3.312024e-09,8.779863e-09,7.400000e-09,2.890853e-09,5.709757e-10,2.171227e-09,6.757242e-10,2.005702e-09,2.861980e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20253,n,0.000005,0.000008,0.000012,0.000007,0.000013,0.000010,0.000018,0.000017,0.000008,...,5.183056e-09,9.379552e-09,1.978565e-08,8.485186e-09,8.522990e-09,3.671670e-09,3.326887e-09,7.048883e-10,1.659072e-09,1.391190e-10
20254,n,0.000035,0.000051,0.000024,0.000011,0.000012,0.000008,0.000008,0.000005,0.000002,...,6.857218e-09,7.720315e-09,7.999804e-09,6.056920e-09,2.237395e-09,4.055173e-09,5.554118e-10,2.438095e-09,7.414557e-10,1.556200e-09
20255,n,0.000008,0.000008,0.000007,0.000037,0.000035,0.000020,0.000028,0.000007,0.000006,...,6.150013e-09,7.243945e-09,5.711502e-09,5.689413e-09,5.574070e-09,3.924516e-09,2.758703e-09,1.446737e-09,8.830650e-10,9.809770e-10
20256,n,0.000021,0.000038,0.000039,0.000020,0.000030,0.000011,0.000009,0.000007,0.000005,...,9.780405e-09,3.613270e-09,7.738320e-10,1.116278e-08,3.097371e-09,3.617548e-09,1.286851e-09,3.688830e-10,2.663532e-09,6.367370e-09


# 🔎&nbsp; <font color="#6DA4AA">Exploring States Data</font>

Let's see the state transitions of the first hour (we know that 1 hour is 3600 / 4 = 900 datapoints):

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

plt.title('Sleep State Transition of the First Hour')
plt.plot(mice_eeg.iloc[:900]['state'], color='#1b9e77');
plt.xticks(np.arange(0, 901, 225), np.arange(0, 915 // 15, 225 // 15))
plt.ylabel('State')
plt.xlabel('Time [min]');

As you can see the mouse is awake excepted for the first 3 datapoints where it is in a non-REM stage of sleep.

Now let's see 8 hours of sleep state transitions at a time where we can see the mouse sleeps.

In [ ]:
plt.figure(figsize=(9, 5))
plt.title('Sample of Sleep State Transition during 8 hours')
plt.plot(mice_eeg.iloc[(900*5):(900*5)+(900*8)]['state'], color='#1b9e77')
plt.xticks(np.arange((900*5),(900*5)+(900*8)+1, 900), np.arange(9))
plt.ylabel('State')
plt.xlabel('Time [h]');

<IPython.core.display.Javascript object>

# 💡&nbsp; <font color='#F5B82E'>Question</font>

<font color='#F5B82E'>1. What's the big difference between this sleep stage transition and the one below of a human (note: here we have multiple non-REM types).</font>

![](https://www.researchgate.net/publication/305754675/figure/fig1/AS:391434457894913@1470336655208/A-hypnogram-showing-normal-distribution-of-sleep-stages.png)

We think, the mouse sleep is very much oscillating between awake and REM states. Whereas human sleep lasts longer in the different phases(N3) and its frequency is not as high as mouses's one.

# 💡&nbsp; <font color='#F5B82E'>Question</font>

<font color='#F5B82E'>2. Look at this bar plot of the states count. Is the dataset balanced?</font>




No, it is not. We can thoroughly see, that the number of observations are not balanced equally between the differents classes of state.

In [ ]:
from collections import Counter

mice_eeg.state.value_counts().plot(kind='bar', cmap='Dark2')
plt.title('Total Occurences per State')
plt.xticks(rotation=0)
plt.ylabel('Count')
plt.xlabel('State');

# 💡&nbsp; <font color='#F5B82E'>Question</font>

<font color='#F5B82E'>3.a For each states, create a list of state durations (consecutive seconds the mouse remain in a state) and create subplots with the boxplots of each of the states durations as a subplot. Make sure this plot as a title, labeled axes and generally look good.</font>

<font color='#F5B82E'>3.b What was the longest time the mouse was awake? And what was the median duration in seconds for each states?</font>


In [ ]:
#3a
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
data = mice_eeg.copy()
data = pd.DataFrame(data)

# Calculate state durations based on row numbers we compute the cumulative sum for each unique state
data['duration'] = data.groupby('state').cumcount() + 1

# Create subplots with boxplots for each unique state
fig, ax = plt.subplots(figsize=(12, 8))
sns.boxplot(x='state', y='duration', data=data, ax=ax)
ax.set_title('State Durations')
ax.set_xlabel('States')
ax.set_ylabel('Duration (datapoints)')

plt.show()


<IPython.core.display.Javascript object>

In [ ]:
#3b

# maximum time spent in awake state
print("longest time the mouse was awake is")
print(f"{max(data.groupby('state').max('duration')['duration']) *4 /3600} hours\n\n")
# longest_awake_duration = data.loc[data['state'] == 'w', 'duration'].max()
# print(longest_awake_duration * 4 / 3600)


# median duration in hours in each state
print("mean duration in each state in hours")
data['duration_sec'] = data['duration'] * 4 /3600
print(f"{data.groupby('state').median('duration_sec')['duration_sec']}")


longest time the mouse was awake is
13.524444444444445 hours


mean duration in each state in hours
state
n    3.745556
r    0.747778
w    6.762778
Name: duration_sec, dtype: float64


# 🔎&nbsp; <font color="#6DA4AA">Exploring Spectrum Data</font>

Here we see the first row frequencies amplitudes. Because rows were generated each 4 seconds, it means that these features have been extracted during the first 4 seconds of the EEG.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(11, 3))
plt.title('Frequency Domain of the First 4 Seconds')
mice_eeg.iloc[0][[f'amplitude_around_{n + 1}_Hertz' for n in range(101)]].plot(kind='bar', cmap="Dark2")
plt.xticks(np.arange(0, 102, 5), np.arange(0, 102, 5), rotation=0)
plt.ylabel('Amplitude')
plt.xlabel('Frequency (Hz)');

<IPython.core.display.Javascript object>



# 💡&nbsp; <font color='#F5B82E'>Question</font>

<font color='#F5B82E'>4. We can see that the highest amplitude is at 4 hertz. What does that mean about the raw signal that was captured during these 4 seconds?</font>


It means that the sin wave with a frequency of 4Hz was the one with the largest amplitude. Even though all frequency were present. T
he function is A * sin(omega * t + phi).

In the plot below we can see boxplots of the frequencies amplitude for each of the sleep state. Note that we don't show the outliers in this plot.


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 8), sharey=True)

fig.suptitle("Boxplots of States Frequencies Amplitudes")

boxplot1 = ax1.boxplot(mice_eeg[mice_eeg['state'] == 'w'].drop('state', axis=1), patch_artist=True, medianprops=dict(color="black", linewidth=1.5), showfliers=False)
ax1.set_xticks(np.arange(0, 102, 5), np.arange(0, 102, 5))
ax1.set_title('awake')

boxplot2 = ax2.boxplot(mice_eeg[mice_eeg['state'] == 'n'].drop('state', axis=1), patch_artist=True, medianprops=dict(color="black", linewidth=1.5), showfliers=False)
ax2.set_xticks(np.arange(0, 102, 5), np.arange(0, 102, 5))
ax2.set_title('non-REM')

boxplot3 = ax3.boxplot(mice_eeg[mice_eeg['state'] == 'r'].drop('state', axis=1), patch_artist=True, medianprops=dict(color="black", linewidth=1.5), showfliers=False)
ax3.set_xticks(np.arange(0, 102, 5), np.arange(0, 102, 5));
ax3.set_title('REM')

fig.supylabel('Amplitude')
fig.supxlabel('Frequencies [Hz]')

# fill with colors
colors = ['#7ED7C1', '#F0DBAF', '#DC8686']
for bplot, color in zip([boxplot1, boxplot2, boxplot3], colors):
    for patch in bplot['boxes']:
        patch.set_facecolor(color)

plt.tight_layout()

<IPython.core.display.Javascript object>

# 💡&nbsp; <font color='#F5B82E'>Question</font>

<font color='#F5B82E'>5.a At which frequencies does the electrical activity mostly occur?</font>

<font color='#F5B82E'>5.b Create boxplots of the first ten frequencies with states boxplots side by side (each states boxplots of the ten feature, then of the second one and so on).</font>

<font color='#F5B82E'>5.c What can you say about the inter- and intra-class variability?</font>

5a:
  The largest activity occurs between 1Hz and 10 Hz.


In [ ]:
# 5b:
data = mice_eeg.copy()

fig, axs = plt.subplots(2, 5, figsize=(25, 15))
axs = axs.flatten()

for i, col in enumerate(data.columns[1:11]):
    sns.boxplot(x='state', y=col, data=data, ax=axs[i])

fig.suptitle('Boxplot of the amplitudes', fontsize=14)
fig.tight_layout(rect=[0, 0.03, 1, 0.98])


<IPython.core.display.Javascript object>

5c:
  The most interesting variability is between 2-6 Hz. Those are the features most likely be used to discriminate the classes of states.

  The greater variability intra class is in wake state
  The smaller std is in r state

# ⚙️&nbsp; <font color="#6DA4AA">Defining Rules Manually to Classify Sleep States</font>

# 💡&nbsp; <font color='#F5B82E'>Question</font>

<font color='#F5B82E'>6.a Choose three frequencies you think are good to separates classes, explain why you chose them and plot a 3d scatter plot with them and the state as the colors.</font>

<font color='#F5B82E'>6.b Are the states easily distinguishable from each other? Explain why.</font>

<font color='#F5B82E'>6.c Create some rules with these three features to classify samples state. Look at the scatter plot and boxplots to help you find rules. Provide your accuracy, f1-score and confusion matrix. (you don't need to do any train/validation separation or aim for high performance)</font>

<font color='#F5B82E'>6.d Create a plot of the states transitions you predicted between the rows slice \[$(900*5):(900*6)$\] (hour 5 to 6 of the datapoints we have) and compare it to the real states transitions.</font>

In [ ]:
# 6a:
# We use our friend plotly to show a breathtaking graph

import plotly.express as px
df=data
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

x = data['amplitude_around_4_Hertz']
y = data['amplitude_around_3_Hertz']
z = data['amplitude_around_8_Hertz']

fig = px.scatter_3d(data, x='amplitude_around_4_Hertz', y='amplitude_around_3_Hertz', z='amplitude_around_8_Hertz',
              color='state')
fig.show()


<IPython.core.display.Javascript object>

In [ ]:

# data.loc[data['state'] == 'w', 'amplitude_around_8_Hertz'].quantile()
# 6b:
# States are distinguable but not perfectly as the data are quiet compact and not equally balanced !

# 6c:
# We choose amplitude @ 4 Hz to discriminate the non REM state [n]
# if amplitude_around_4_Hertz > 1.691593e-05 then -> n state
# # We choose amplitude @ 3 and 8 Hz to discriminate the REM state [r]
# if ampl around 3 Hz < 5.059523e-06 AND ampl around 8 Hz > 1.691593e-05 then-> r state
# else w state
def class_state(df):
    if df['amplitude_around_4_Hertz'] >= 1.691593e-05:
        return 'n'
    elif df['amplitude_around_3_Hertz'] <= 5.059523e-06 and df['amplitude_around_8_Hertz'] >= 1.691593e-05:
        return 'r'
    else:
        return 'w'

data['pred_state'] = data.apply(class_state, axis=1)

# accuracy
# F1-score
# conf-matrix

from sklearn import metrics

score = accuracy_score(data['state'], data['pred_state'])
f1_s = f1_score(data['state'], data['pred_state'], average=None)

print(f"Accuracy: {score}")
print(f"F1 score: {f1_s}")

# confusion matrix not showing in Google Colab do not know why
cm = metrics.confusion_matrix(data["state"], data['pred_state'])
disp = metrics.ConfusionMatrixDisplay(cm, display_labels = ['n', 'r', 'w'])

disp.plot()
plt.show()



Accuracy: 0.7160134267943529
F1 score: [0.62560885 0.24707135 0.79049953]


<IPython.core.display.Javascript object>

In [ ]:
# 6d:
# So we compute the scores at the requested time h5 to h6

d=data.copy()
d=d.iloc[900*5 : 900*6]


score = accuracy_score(d['state'], d['pred_state'])
f1_s = f1_score(d['state'], d['pred_state'], average=None)

print(f"Accuracy: {score}")
print(f"F1 score: {f1_s}")

# confusion matrix not showing in Google Colab do not know why
cm = metrics.confusion_matrix(d["state"], d['pred_state'])
disp = metrics.ConfusionMatrixDisplay(cm, display_labels = ['n', 'r', 'w'])

disp.plot()
plt.show()

# we see that in this time scope there may not be r state

Accuracy: 0.86
F1 score: [0.7921147  0.         0.89772727]


<IPython.core.display.Javascript object>